# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import matplotlib.pyplot as plt

import numpy as np
import random

# Reading data

In [3]:
df = pd.read_csv('data/fpl_fbref_elo_players.csv')

In [4]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
df.columns

Index(['Date', 'Day', 'Comp', 'Round', 'Venue', 'Squad', 'Opponent', 'Start',
       'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR',
       'Touches', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xAG', 'SCA', 'GCA',
       'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries', 'PrgC', 'Att.1', 'Succ',
       'Match Report', 'Name', 'Season', 'Fls', 'Fld', 'Off', 'Crs', 'TklW',
       'OG', 'PKwon', 'PKcon', 'WDL', 'GoalsTeams', 'Team Score', 'Opp Score',
       'CS', 'Was Home', 'GW', 'neutral_name', 'Total Points', 'FPL position',
       'Opp rating', 'Team rating'],
      dtype='object')

# New features

In [6]:
# average goals for and against team in last 5 matches
grouped = df.groupby('Name')

In [7]:
df['ScoreForLast5'] = grouped['Team Score'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

In [8]:
df['ScoreAgainstLast5'] = grouped['Opp Score'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

In [9]:
df['ScoreForLast5'] = df['ScoreForLast5'].shift(1)
df['ScoreAgainstLast5'] = df['ScoreAgainstLast5'].shift(1)

In [10]:
df["Min_points"] = (df["Min"] > 0) + (df["Min"] > 60)

In [ ]:
%%time
# calculates the average of a feature for all fixtures that took place before that game
overall_average_columns = ["Min", "Gls", "Ast", "CrdY", "CrdR", "xG", "xAG", "CS", "Team Score", "Opp Score"]

for i, row in df.iterrows():
    name = row["Name"]
    date = row["Date"]
    for col in overall_average_columns:
        df.loc[i, col + "AvgOverall"] = df[(df["Date"] < date) & (df["Name"] == name)][col].mean()

In [ ]:
df[df["Name"] == "Mohamed-Salah"][["ScoreForLast5", "ScoreAgainstLast5", "Team Score", "Opp Score", "GlsAvgOverall", "Gls"]]

In [ ]:
df.columns

# Saving df to file

In [ ]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
df.to_csv('data/final_dataset.csv')

In [ ]:
df